In [1]:
import sys 
import os 
sys.path.append("..")
os.chdir("..")

# Plots

# Plot Single Result

In [2]:
import pandas as pd
from metrics.tasks import TaskType
from models.models import ModelType
from datasets.transforms import TransformType

In [3]:
# result to be loaded
task_type = TaskType.NAME
model_type = ModelType.MLP
transform_type = TransformType.degree_transform_onehot
result_path = f"./.ignore_temp_{task_type.name.lower()}.csv"

# load results
df = pd.read_csv(result_path) 
indeces = lambda types:  (df['type_model'] == types[0].name.lower()) & (df['transform'] == types[1].name.lower())
filtered_results = df[indeces((model_type, transform_type))]
filtered_results

,test_loss,test_Accuracy,test_BalancedAccuracy,type_model,transform
15,0.087076,0.866481,0.767130,mlp,degree_transform_onehot
16,0.078043,0.699699,0.647036,mlp,degree_transform_onehot
17,0.074661,0.702712,0.703165,mlp,degree_transform_onehot
18,0.098400,0.724965,0.697048,mlp,degree_transform_onehot
19,0.100822,0.767617,0.682374,mlp,degree_transform_onehot


In [4]:
metric = "test_BalancedAccuracy"
mean = filtered_results[metric].mean()
std = filtered_results[metric].std()
min_ = filtered_results[metric].min()
max_ = filtered_results[metric].max()

print(f"Task: {task_type.name.lower()}, model: {model_type.name.lower()}, transform: {transform_type.name.lower()}")
print(f"{metric}\nMEAN: {mean}\nSTD: {std}\nMIN: {min_}\nMAX: {max_}")

Task: name, model: mlp, transform: degree_transform_onehot
test_BalancedAccuracy
MEAN: 0.6993504166603088
STD: 0.043700399514380414
MIN: 0.6470359563827515
MAX: 0.7671297788619995


In [5]:
# utils

def divider_up():
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ")
def divider_down():
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< \n\n")

def divider_up_s():
    print("+++++++++++++++")
def divider_down_s():
    print("---------------\n")

def divider_up_xs():
    print("```")
def divider_down_xs():
    print("'''")



models = [ModelType.GAT, ModelType.GCN, ModelType.MLP, ModelType.TransfConv]
transforms = [TransformType.degree_transform_onehot, TransformType.random_node_features, TransformType.degree_transform]

get_result_path = lambda task_type: f"./results_{task_type.name.lower()}.csv"


## Average Performance Metrics

- Max performance of a configuration over all runs
- Max performance over preprocessing transforms
- Average over models

In [6]:
import numpy as np 

for task_type in TaskType:
    print(f"TASK: {task_type.name}")
    divider_up()
    result_path = get_result_path(task_type)
    df = pd.read_csv(result_path) 
    indeces = lambda types:  (df['type_model'] == types[0].name.lower()) & (df['transform'] == types[1].name.lower())
    
    metric_columns = [col for col in df[(df['type_model'] == task_type.name.lower())].columns if col.startswith("test_")]

    for metric in metric_columns:

        metric_results = []
        
        for model_type in models:

            model_results = []
            for transform_type in transforms:
                filtered_results = df[indeces((model_type, transform_type))]
                max_ = filtered_results[metric].max()                
                model_results.append(max_)
            
            metric_results.append(np.max(model_results))
        
        res = np.mean(metric_results)
        if metric != "test_loss":
            print(f"Metric: {metric}; Mean: {res}")
    divider_down()

            

TASK: NAME
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
Metric: test_Accuracy; Mean: 0.8437065333127975
Metric: test_BalancedAccuracy; Mean: 0.6532842516899109
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 


TASK: ORIENTABILITY
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
Metric: test_F1Score; Mean: 0.40389081090688705
Metric: test_MCC; Mean: 0.0
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 


TASK: BETTI_NUMBERS
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
Metric: test_betti_0_Accuracy; Mean: 1.0
Metric: test_betti_1_Accuracy; Mean: 0.928720474243164
Metric: test_betti_1_BalancedAccuracy; Mean: 0.9382774531841278
Metric: test_betti_2_Accuracy; Mean: 0.9208101481199263
Metric: test_betti_2_MCC; Mean: 0.062067294493317576
Metric: test_betti_2_BalancedAccuracy; Mean: 0.5091036856174469
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 




## Model Performance Metrics

- Per task
- Per metric
- Per model
- Max over runs
- Statistics over transforms

In [7]:
import numpy as np 

for task_type in TaskType:
    print(f"TASK: {task_type.name}")
    divider_up()
    result_path = f"./results_{task_type.name.lower()}.csv"
    df = pd.read_csv(result_path) 
    indeces = lambda types:  (df['type_model'] == types[0].name.lower()) & (df['transform'] == types[1].name.lower())
    
    metric_columns = [col for col in df[(df['type_model'] == task_type.name.lower())].columns if (col.startswith("test_") and col != "test_loss" ) ]

    for metric in metric_columns:
        print(f"METRIC: {metric}")
        divider_up_s()
        metric_results = []
        
        for model_type in [ModelType.GAT, ModelType.GCN, ModelType.MLP, ModelType.TransfConv, ModelType.TAG]:

            model_results = []

            for transform_type in [TransformType.degree_transform_onehot, TransformType.random_node_features, TransformType.degree_transform]:
                filtered_results = df[indeces((model_type, transform_type))]
                model_results.append(filtered_results[metric].max() )
            
            print(f"Model: {model_type.name.lower()}; Best: {np.max(model_results)}; Std: {np.std(model_results)}")
        
        divider_down_s()

    divider_down()

TASK: NAME
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
METRIC: test_Accuracy
+++++++++++++++
Model: gat; Best: 0.8011126518249512; Std: 0.28651992969309925
Model: gcn; Best: 0.8566295504570007; Std: 0.02290385734273917
Model: mlp; Best: 0.8920955061912537; Std: 0.03804452399717796
Model: transfconv; Best: 0.8249884247779846; Std: 0.3123100601847973
Model: tag; Best: 0.8592953085899353; Std: 0.023560905994148878
---------------

METRIC: test_BalancedAccuracy
+++++++++++++++
Model: gat; Best: 0.6288862228393555; Std: 0.16764334927536534
Model: gcn; Best: 0.6573934555053711; Std: 0.19949997924506085
Model: mlp; Best: 0.7762631177902222; Std: 0.2588038470717644
Model: transfconv; Best: 0.5505942106246948; Std: 0.1520862830963586
Model: tag; Best: 0.5454341769218445; Std: 0.14521965917839316
---------------

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 


TASK: ORIENTABILITY
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
METRIC: test_F1

## Model Performance Metrics

- Per task
- Per metric
- Per Model
- Per Transform
- Statistics over runs


In [8]:
import numpy as np 

for task_type in TaskType:
    print(f"TASK: {task_type.name}")
    divider_up()
    result_path = f"./results_{task_type.name.lower()}.csv"
    df = pd.read_csv(result_path) 
    indeces = lambda types:  (df['type_model'] == types[0].name.lower()) & (df['transform'] == types[1].name.lower())
    
    metric_columns = [col for col in df[(df['type_model'] == task_type.name.lower())].columns if (col.startswith("test_") and col != "test_loss" ) ]

    for metric in metric_columns:
        print(f"METRIC: {metric}")
        divider_up_s()
        metric_results = []
        
        for model_type in [ModelType.GAT, ModelType.GCN, ModelType.MLP, ModelType.TransfConv, ModelType.TAG]:
            print(f"MODEL: {model_type.name.lower()}")
            divider_up_xs()
            model_results = []

            for transform_type in [TransformType.degree_transform_onehot, TransformType.random_node_features, TransformType.degree_transform]:
                filtered_results = df[indeces((model_type, transform_type))]
                model_results.append(filtered_results[metric].max() )
            
                print(f"Transform: {transform_type.name.lower()}; Best: {filtered_results[metric].max()}; Std: {filtered_results[metric].std()}")

            divider_down_xs()
        divider_down_s()

    divider_down()

TASK: NAME
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
METRIC: test_Accuracy
+++++++++++++++
MODEL: gat
```
Transform: degree_transform_onehot; Best: 0.1030366271734237; Std: 0.006215618301668605
Transform: random_node_features; Best: 0.8011126518249512; Std: 0.02083502535766087
Transform: degree_transform; Best: 0.3893138766288757; Std: 0.13546894412099425
'''
MODEL: gcn
```
Transform: degree_transform_onehot; Best: 0.8193092346191406; Std: 0.3664916820571175
Transform: random_node_features; Best: 0.8016921877861023; Std: 0.0
Transform: degree_transform; Best: 0.8566295504570007; Std: 0.23450474055846815
'''
MODEL: mlp
```
Transform: degree_transform_onehot; Best: 0.8664812445640564; Std: 0.06936321306766678
Transform: random_node_features; Best: 0.8016921877861023; Std: 0.0
Transform: degree_transform; Best: 0.8920955061912537; Std: 0.011444242433786535
'''
MODEL: transfconv
```
Transform: degree_transform_onehot; Best: 0.1436022222042083; Std: 0.01488008183202512